In [12]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from proj1_helpers import load_csv_data
from functions import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**1.   SETUP THE DATA**

We create pandas DataFrames to clear the train data and the test data.

In [164]:
train_set_pd = pd.read_csv('Data/train.csv')

In [165]:
test_set_pd = pd.read_csv('Data/test.csv')

Separate the data depending on the columns used which can be found by looking at the parameter 'PRI_jet_num'

In [15]:
train_set_pd_0 = train_set_pd[train_set_pd.PRI_jet_num == 0].drop(['DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_lep_eta_centrality', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_num'], axis = 1)
train_set_pd_1 = train_set_pd[train_set_pd.PRI_jet_num == 1].drop(['DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_lep_eta_centrality', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_num'], axis = 1)
train_set_pd_2 = train_set_pd[train_set_pd.PRI_jet_num.isin([2, 3])].drop(['PRI_jet_num'], axis = 1)

In [16]:
test_set_pd_0 = test_set_pd[test_set_pd.PRI_jet_num == 0].drop(['DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_lep_eta_centrality', 'PRI_jet_leading_pt', 'PRI_jet_leading_eta', 'PRI_jet_leading_phi', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_num'], axis = 1)
test_set_pd_1 = test_set_pd[test_set_pd.PRI_jet_num == 1].drop(['DER_deltaeta_jet_jet', 'DER_mass_jet_jet', 'DER_prodeta_jet_jet', 'DER_lep_eta_centrality', 'PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'PRI_jet_num'], axis = 1)
test_set_pd_2 = test_set_pd[test_set_pd.PRI_jet_num.isin([2, 3])].drop(['PRI_jet_num'], axis = 1)

We replace the -999 by the mean of the column.

In [17]:
mean_0_tr = get_column_mean(-999.000, train_set_pd_0, 'DER_mass_MMC')
mean_1_tr = get_column_mean(-999.000, train_set_pd_1, 'DER_mass_MMC')
mean_2_tr = get_column_mean(-999.000, train_set_pd_2, 'DER_mass_MMC')

train_set_pd_0['DER_mass_MMC'] = train_set_pd_0['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_0_tr, 'DER_mass_MMC'))
train_set_pd_1['DER_mass_MMC'] = train_set_pd_1['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_1_tr, 'DER_mass_MMC'))
train_set_pd_2['DER_mass_MMC'] = train_set_pd_2['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_2_tr, 'DER_mass_MMC'))

In [18]:
mean_0_te = get_column_mean(-999.000, test_set_pd_0, 'DER_mass_MMC')
mean_1_te = get_column_mean(-999.000, test_set_pd_1, 'DER_mass_MMC')
mean_2_te = get_column_mean(-999.000, test_set_pd_2, 'DER_mass_MMC')

test_set_pd_0['DER_mass_MMC'] = test_set_pd_0['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_0_te, 'DER_mass_MMC'))
test_set_pd_1['DER_mass_MMC'] = test_set_pd_1['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_1_te, 'DER_mass_MMC'))
test_set_pd_2['DER_mass_MMC'] = test_set_pd_2['DER_mass_MMC'].apply(lambda x: replace_by_mean(x, mean_2_te, 'DER_mass_MMC'))

Export the three DataFrames then we can open them with the load_csv function, which returns usable tuples of y's, x's and ids

In [59]:
train_set_pd_0.to_csv('Data/train_set_0.csv', index = False)
train_set_pd_1.to_csv('Data/train_set_1.csv', index = False)
train_set_pd_2.to_csv('Data/train_set_2.csv', index = False)

In [60]:
test_set_pd_0.to_csv('Data/test_set_0.csv', index = False)
test_set_pd_1.to_csv('Data/test_set_1.csv', index = False)
test_set_pd_2.to_csv('Data/test_set_2.csv', index = False)

In [61]:
train_set_0 = load_csv_data('Data/train_set_0.csv', sub_sample=True)
train_set_1 = load_csv_data('Data/train_set_1.csv', sub_sample=True)
train_set_2 = load_csv_data('Data/train_set_2.csv', sub_sample=True)

In [107]:
test_set_0 = load_csv_data('Data/test_set_0.csv', sub_sample=False)
test_set_1 = load_csv_data('Data/test_set_1.csv', sub_sample=False)
test_set_2 = load_csv_data('Data/test_set_2.csv', sub_sample=False)

We separate x's the y's and the id's

In [166]:
x_train_0 = train_set_0[1]
x_train_1 = train_set_1[1]
x_train_2 = train_set_2[1]

y_train_0 = train_set_0[0]
y_train_1 = train_set_1[0]
y_train_2 = train_set_2[0]

In [167]:
x_test_0 = test_set_0[1]
x_test_1 = test_set_1[1]
x_test_2 = test_set_2[1]

id_test_0 = test_set_0[2]
id_test_1 = test_set_1[2]
id_test_2 = test_set_2[2]

We standardize all observations

In [168]:
#Standardized version
x_train_0 = (x_train_0.T - x_train_0.mean(1)).T
y_0 = x_train_0.std(1)
x_train_0 = (x_train_0.T/y_0).T
x_train_0 = np.column_stack((np.ones(len(x_train_0)), x_train_0))

x_train_1 = (x_train_1.T - x_train_1.mean(1)).T
y_1 = x_train_1.std(1)
x_train_1 = (x_train_1.T/y_1).T
x_train_1 = np.column_stack((np.ones(len(x_train_1)), x_train_1))

x_train_2 = (x_train_2.T - x_train_2.mean(1)).T
y_2 = x_train_2.std(1)
x_train_2 = (x_train_2.T/y_2).T
x_train_2 = np.column_stack((np.ones(len(x_train_2)), x_train_2))

In [169]:
#Standardized version
x_test_0 = (x_test_0.T - x_test_0.mean(1)).T
y_te_0 = x_test_0.std(1)
x_test_0 = (x_test_0.T/y_te_0).T
x_test_0 = np.column_stack((np.ones(len(x_test_0)), x_test_0))

x_test_1 = (x_test_1.T - x_test_1.mean(1)).T
y_te_1 = x_test_1.std(1)
x_test_1 = (x_test_1.T/y_te_1).T
x_test_1 = np.column_stack((np.ones(len(x_test_1)), x_test_1))

x_test_2 = (x_test_2.T - x_test_2.mean(1)).T
y_te_2 = x_test_2.std(1)
x_test_2 = (x_test_2.T/y_te_2).T
x_test_2 = np.column_stack((np.ones(len(x_test_2)), x_test_2))

We replace all "-1" prediction by "0"

In [113]:
y_train_0 = neg_to_zero(y_train_0)
y_train_1 = neg_to_zero(y_train_1)
y_train_2 = neg_to_zero(y_train_2)

**2. DEFINE ALL FUNCTION **

In [114]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    w = initial_w
    for n_iter in range(max_iters):
        gradient = compute_gradient_least_square(y, tx, w)
        loss = compute_mse(y, tx, w)
        w = w - gamma*gradient
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return loss, w

In [115]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma):
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):    
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            gradient = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
            loss = compute_mse(minibatch_y, minibatch_tx, w)
            new_w = w - gamma*gradient
            w = new_w
            ws.append(w)
            losses.append(loss)
            print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
            bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    return losses, ws

In [116]:
def least_squares(y, tx):
    A = tx.T@tx
    b = tx.T@y
    w = np.linalg.solve(A, b)
    loss = compute_mse(y, tx, w)
    return w, loss

In [117]:
def ridge_regression(y, tx, lambda_):
    X = tx.T@tx
    N = X.shape[0]
    A = (X + 2*N*lambda_*np.identity(N))
    b = tx.T@y
    w = np.linalg.solve(A, b)
    loss = compute_mse(y, tx, w)
    return w, loss

In [118]:
def logistic_regression(y, tx, initial_w, max_iters, gamma):
    loss, w = stoch_gradient_descent_log_reg(y, tx, initial_w, max_iters, gamma)
    return w, loss

In [119]:
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    return w, loss

**3. GET THE MODEL**

We make some tests to find the correct gamma

In [212]:
gammas = np.logspace(-5, 0)
losses_0 = []
losses_1 = []
losses_2 = []
for gamma in gammas:
    w_0, loss_0 = logistic_regression(y_train_0, x_train_0, np.zeros(x_train_0.shape[1]), 1000, gamma)
    w_1, loss_1 = logistic_regression(y_train_1, x_train_1, np.zeros(x_train_1.shape[1]), 1000, gamma)
    w_2, loss_2 = logistic_regression(y_train_2, x_train_2, np.zeros(x_train_2.shape[1]), 1000, gamma)
    
    losses_0.append((gamma, loss_0))
    losses_1.append((gamma, loss_1))
    losses_2.append((gamma, loss_2))

In [213]:
losses_0.sort(key = lambda x: x[1])
l_0  = np.take(losses_0, [x for x in range(20)], axis = 0)
gamma_0 = l_0.mean(axis = 0)[0]

losses_1.sort(key = lambda x: x[1])
l_1  = np.take(losses_1, [x for x in range(20)], axis = 0)
gamma_1 = l_1.mean(axis = 0)[0]

losses_2.sort(key = lambda x: x[1])
l_2  = np.take(losses_2, [x for x in range(20)], axis = 0)
gamma_2 = l_2.mean(axis = 0)[0]

In [214]:
gamma_0, gamma_1, gamma_2

(0.039085977227429299, 0.016980548002622053, 0.021153959536203651)

We commpute the model and the log for the 3 different sets of observation

In [215]:
w_0, loss_0 = logistic_regression(y_train_0, x_train_0, np.zeros(x_train_0.shape[1]), 10000, gamma_0)

In [216]:
w_1, loss_1 = logistic_regression(y_train_1, x_train_1, np.zeros(x_train_1.shape[1]), 10000, gamma_1)

In [217]:
w_2, loss_2 = logistic_regression(y_train_2, x_train_2, np.zeros(x_train_2.shape[1]), 10000, gamma_2)

**4. TEST ON TEST SET**

In [218]:
y_0 = zero_to_neg(np.around(sigmoid(x_test_0 @ w_0)))
y_1 = zero_to_neg(np.around(sigmoid(x_test_1 @ w_1)))
y_2 = zero_to_neg(np.around(sigmoid(x_test_2 @ w_2)))

s_0 = np.column_stack((id_test_0, y_0))
s_1 = np.column_stack((id_test_1, y_1))
s_2 = np.column_stack((id_test_2, y_2))
s = np.vstack((np.vstack((s_0, s_1)), s_2))
s = sorted(s, key=lambda x: x[0])

In [161]:
s_df = pd.DataFrame(s)
s_df.columns = ['Id', 'Prediction']
s_df.Id = s_df.Id.apply(lambda x: int(x))
s_df.Prediction = s_df.Prediction.apply(lambda x: int(x))

In [223]:
s_df.to_csv('Data/4_submit_prediction_st_ones_pr.csv', index=False)